In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import xlrd

# here you have to enter your actual API key from SimFin
api_key = "khJmsEQpgG1bK7Xl1vgXwyv9SCbrsBSP"

# list of tickers we want to get data for
tickers = ["AAPL",
            "NVDA",
            "WMT",
            "MSFT",
            "GOOG",
            "AMZN",
            "FB",
            "BRK.B",
            "JPM",
            "V",
            "JNJ",
            "XOM",
            "BAC",
            "PG",
            "T",
            "MA",
            "HD",
            "INTC",
            "DIS",
            "VZ",
            "UNH"]

# first: find SimFin IDs
sim_ids = []
for ticker in tickers:

    request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
    content = requests.get(request_url)
    data = content.json()

    if "error" in data or len(data) < 1:
        sim_ids.append(None)
    else:
        sim_ids.append(data[0]['simId'])

print(sim_ids)

# define time periods for financial statement data
statement_type = "bs"
time_periods = ["Q1", "Q2", "Q3", "Q4"]
year_start = 2015
year_end = 2019

# set up the XLSX writer
writer = pd.ExcelWriter("simfin_data_bs.xlsx", engine='xlsxwriter')

data = {}
# get data for each ticker/id
for idx, sim_id in enumerate(sim_ids):
    d = data[tickers[idx]] = {"Line Item": []}
    if sim_id is not None:
        for year in range(year_start, year_end + 1):
            for time_period in time_periods:

                # make time period identifier
                period_identifier = time_period + "-" + str(year)

                if period_identifier not in d:
                    d[period_identifier] = []

                request_url = f'https://simfin.com/api/v1/companies/id/{sim_id}/statements/standardised?stype={statement_type}&fyear={year}&ptype={time_period}&api-key={api_key}'

                content = requests.get(request_url)
                statement_data = content.json()

                # collect line item names once, they are the same for all companies with the standardised data
                if len(d['Line Item']) == 0 and 'values' in statement_data:
                    d['Line Item'] = [x['standardisedName'] for x in statement_data['values']]

                if 'values' in statement_data:
                    for item in statement_data['values']:
                        d[period_identifier].append(item['valueChosen'])
                else:
                    # no data found for time period
                    d[period_identifier] = [None for _ in d['Line Item']]

        # fix the periods where no values were available
        len_target = len(d['Line Item'])
        if len_target > 0:
            for k, v in d.items():
                if len(v) != len_target:
                    d[k] = [None for _ in d['Line Item']]

    # convert to pandas dataframe
    df = pd.DataFrame(data=d)
    df["symbol"]=tickers[idx]
    # save in the XLSX file configured earlier
    df.to_excel(writer, sheet_name=tickers[idx])#,index=False)#get rid of the unnamed ON THE CONCAT

writer.save()
writer.close()

[111052, 172199, 239962, 59265, 18, 62747, 121021, None, 243510, 107326, 89661, 121214, 71941, 133209, 66790, 331628, 261673, 85652, 261356, 101219, 367714]


PermissionError: [Errno 13] Permission denied: 'simfin_data_bs.xlsx'

In [2]:
loc=("../project_22/simfin_data_bs.xlsx")
wb=xlrd.open_workbook(loc) #basically a pd.csv

In [3]:
df.to_csv(r'C:\Users\roel Cantu\Desktop\project_2.csv')
df.head()

,Line Item,Q1-2015,Q2-2015,Q3-2015,Q4-2015,Q1-2016,Q2-2016,Q3-2016,Q4-2016,Q1-2017,...,Q4-2017,Q1-2018,Q2-2018,Q3-2018,Q4-2018,Q1-2019,Q2-2019,Q3-2019,Q4-2019,symbol
0,"Cash, Cash Equivalents & Short Term Investments",10430000000,9802000000,9925000000,12911000000,13014000000,10974000000,15667000000,13275000000,19352000000,...,15490000000,22041000000,21860000000,13849000000,14324000000,15710000000,None,None,None,UNH
1,Cash & Cash Equivalents,8650000000,7887000000,7983000000,10923000000,10596000000,8217000000,12796000000,10430000000,16142000000,...,11981000000,18243000000,18368000000,10263000000,10866000000,12407000000,None,None,None,UNH
2,Short Term Investments,1780000000,1915000000,1942000000,1988000000,2418000000,2757000000,2871000000,2845000000,3210000000,...,3509000000,3798000000,3492000000,3586000000,3458000000,3303000000,None,None,None,UNH
3,Accounts & Notes Receivable,5040000000,6034000000,6752000000,6523000000,7777000000,9346000000,7347000000,8152000000,9595000000,...,9568000000,11512000000,10874000000,10992000000,11388000000,12826000000,None,None,None,UNH
4,"Accounts Receivable, Net",5040000000,6034000000,6752000000,6523000000,7777000000,9346000000,7347000000,8152000000,9595000000,...,9568000000,11512000000,10874000000,10992000000,11388000000,12826000000,None,None,None,UNH


In [ ]:
pd.concat(pd.read_excel("simfin_data_bs.xlsx",sheet_name=None), ignore_index=True, sort=True)
df.head()

In [ ]:
df.set_index("Line Item", inplace=True)


In [ ]:
AAPL_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="AAPL")
AAPL_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
AAPL_df.drop(["a"], axis=1, inplace=True)
AAPL_df=AAPL_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
AAPL_df.set_index("Line Item",inplace=True)
AAPL_df.drop(columns= "symbol",inplace=True)
AAPL_df=AAPL_df.T
AAPL_df.head(2)           


In [ ]:
NVDA_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="NVDA")
NVDA_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
NVDA_df.drop(["a"], axis=1, inplace=True)
NVDA_df=NVDA_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
NVDA_df.set_index("Line Item",inplace=True)
NVDA_df.drop(columns= "symbol",inplace=True)
NVDA_df=NVDA_df.T
NVDA_df.head(2)

In [4]:
WMT_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="WMT")
WMT_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
WMT_df.drop(["a"], axis=1, inplace=True)
WMT_df=WMT_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
WMT_df.set_index("Line Item",inplace=True)
WMT_df.drop(columns= "symbol",inplace=True)
WMT_df=WMT_df.T
WMT_df.head(2)

Line Item,"Cash, Cash Equivalents & Short Term Investments",Cash & Cash Equivalents,Accounts & Notes Receivable,"Accounts Receivable, Net",Inventories,Other Short Term Assets,Prepaid Expenses,Total Current Assets,"Property, Plant & Equipment, Net",Other Long Term Assets,...,Total Liabilities,Share Capital & Additional Paid-In Capital,Common Stock,Additional Paid in Capital,Retained Earnings,Other Equity,Equity Before Minority Interest,Minority Interest,Total Equity,Total Liabilities & Equity
Q1-2015,7.759000e+09,7.759000e+09,5.813000e+09,5.813000e+09,4.631000e+10,2.251000e+09,2.251000e+09,6.213300e+10,1.121510e+11,2.646300e+10,...,1.207570e+11,2.677000e+09,323000000.0,2.354000e+09,8.249200e+10,-8.595000e+09,7.657400e+10,3.416000e+09,7.999000e+10,2.007470e+11
Q2-2015,5.751000e+09,5.751000e+09,5.275000e+09,5.275000e+09,4.500700e+10,2.099000e+09,2.099000e+09,5.813200e+10,1.128240e+11,2.766400e+10,...,1.164960e+11,2.300000e+09,321000000.0,1.979000e+09,8.495900e+10,-8.388000e+09,7.887100e+10,3.253000e+09,8.212400e+10,1.986200e+11


In [10]:
MSFT_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="MSFT")
MSFT_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
MSFT_df.drop(["a"], axis=1, inplace=True)
MSFT_df=MSFT_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
MSFT_df.set_index("Line Item",inplace=True)
MSFT_df.drop(columns= "symbol",inplace=True)
MSFT_df=MSFT_df.T
MSFT_df.head(2)

Line Item,"Cash, Cash Equivalents & Short Term Investments",Cash & Cash Equivalents,Short Term Investments,Accounts & Notes Receivable,"Accounts Receivable, Net",Inventories,Other Short Term Assets,Miscellaneous Short Term Assets,Total Current Assets,"Property, Plant & Equipment, Net",...,Miscellaneous Long Term Liabilities,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Common Stock,Retained Earnings,Other Equity,Equity Before Minority Interest,Total Equity,Total Liabilities & Equity
Q1-2015,8.919300e+10,6.302000e+09,8.289100e+10,1.288700e+10,1.288700e+10,3.141000e+09,7.218000e+09,5.434000e+09,1.124390e+11,1.322900e+10,...,1.178100e+10,3.479200e+10,7.948600e+10,6.836200e+10,6.836200e+10,1.805100e+10,3.757000e+09,9.017000e+10,9.017000e+10,1.696560e+11
Q2-2015,9.024900e+10,6.426000e+09,8.382300e+10,1.618600e+10,1.618600e+10,2.053000e+09,7.874000e+09,6.173000e+09,1.163620e+11,1.360700e+10,...,1.242300e+10,3.555400e+10,8.296900e+10,6.876500e+10,6.876500e+10,1.973100e+10,3.383000e+09,9.187900e+10,9.187900e+10,1.748480e+11


In [24]:

GOOG_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="GOOG")
GOOG_df.rename({"Unnamed: 0":"Line Item"}, axis="columns", inplace=True)
GOOG_df=GOOG_df.dropna(axis='columns', how="all").dropna(axis="rows", how="any")
GOOG_df.drop(GOOG_df.columns[1:3],axis=1, inplace=True)
#GOOG_df.set_index("Line Item",inplace=True)
#GOOG_df = df.iloc[1:]
#GOOG_df=GOOG_df.T
GOOG_df

,Line Item,Q3 '15,Q4 '15,Q1 '16,Q2 '16,Q3 '16,Q4 '16,Q1 '17,Q2 '17,Q3 '17,Q4 '17,Q1 '18,Q2 '18,Q3 '18
0,"Cash, Cash Equivalents & Short Term Investments",75264.0,78460.0,83056.0,86333.0,92439.0,94713.0,100143.0,101871.0,102885.0,102254.0,106416.0,109140.0,113488.0
1,Accounts & Notes Receivable,10818.0,11686.0,11979.0,14137.0,12913.0,14003.0,15295.0,18336.0,16777.0,17043.0,17897.0,20838.0,19149.0
3,Other Short Term Assets,4873.0,4092.0,2952.0,4670.0,3162.0,3272.0,3142.0,3352.0,3463.0,4162.0,4177.0,4591.0,4517.0
4,Total Current Assets,90955.0,94238.0,98546.0,105408.0,108794.0,112386.0,119345.0,124308.0,123761.0,124157.0,129702.0,135676.0,138207.0
5,"Property, Plant & Equipment, Net",30162.0,31413.0,32753.0,34234.0,35936.0,37676.0,40120.0,42383.0,48845.0,51672.0,55300.0,59719.0,60528.0
6,Long Term Investments & Receivables,5577.0,5820.0,5705.0,5878.0,6131.0,6642.0,7269.0,7813.0,10976.0,11487.0,12673.0,13859.0,14474.0
7,Other Long Term Assets,23053.0,22821.0,22944.0,21977.0,21895.0,21917.0,22802.0,22791.0,23353.0,24294.0,23863.0,23538.0,32140.0
8,Total Noncurrent Assets,58792.0,60054.0,61402.0,62089.0,63962.0,66235.0,70191.0,72987.0,83174.0,87453.0,91836.0,97116.0,107142.0
9,Total Assets,149747.0,154292.0,159948.0,167497.0,172756.0,178621.0,189536.0,197295.0,206935.0,211610.0,221538.0,232792.0,245349.0
11,Payables & Accruals,11359.0,12157.0,13400.0,15657.0,14108.0,17459.0,19424.0,22751.0,22469.0,28189.0,29549.0,32836.0,33243.0


In [ ]:
AMZN_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="AMZN")
AMZN_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
AMZN_df.drop(["a"], axis=1, inplace=True)
AMZN_df=AMZN_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
AMZN_df.set_index("Line Item",inplace=True)
AMZN_df.drop(columns= "symbol",inplace=True)
AMZN_df=AMZN_df.T
AMZN_df.head(2)

In [ ]:
FB_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="FB")
FB_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
FB_df.drop(["a"], axis=1, inplace=True)
FB_df=FB_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
FB_df.set_index("Line Item",inplace=True)
FB_df.drop(columns= "symbol",inplace=True)
FB_df=FB_df.T
FB_df.head(2)

In [ ]:
JPM_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="JPM")
JPM_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
JPM_df.drop(["a"], axis=1, inplace=True)
JPM_df=JPM_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
JPM_df.set_index("Line Item",inplace=True)
JPM_df.drop(columns= "symbol",inplace=True)
JPM_df=JPM_df.T
JPM_df.head(2)

In [ ]:
V_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="V")
V_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
V_df.drop(["a"], axis=1, inplace=True)
V_df=V_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
V_df.set_index("Line Item",inplace=True)
V_df.drop(columns= "symbol",inplace=True)
V_df=V_df.T
V_df.head(2)

In [ ]:
JNJ_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="JNJ")
JNJ_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
JNJ_df.drop(["a"], axis=1, inplace=True)
JNJ_df=JNJ_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
JNJ_df.set_index("Line Item",inplace=True)
JNJ_df.drop(columns= "symbol",inplace=True)
JNJ_df=JNJ_df.T
JNJ_df.head(2)

In [ ]:
XOM_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="XOM")
XOM_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
XOM_df.drop(["a"], axis=1, inplace=True)
XOM_df=XOM_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
XOM_df.set_index("Line Item",inplace=True)
XOM_df.drop(columns= "symbol",inplace=True)
XOM_df=XOM_df.T
XOM_df.head(2)

In [ ]:
BAC_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="BAC")
BAC_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
BAC_df.drop(["a"], axis=1, inplace=True)
BAC_df=BAC_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
BAC_df.set_index("Line Item",inplace=True)
BAC_df.drop(columns= "symbol",inplace=True)
BAC_df=BAC_df.T
BAC_df.head(2)

In [ ]:
PG_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="PG")
PG_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
PG_df.drop(["a"], axis=1, inplace=True)
PG_df=PG_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
PG_df.set_index("Line Item",inplace=True)
PG_df.drop(columns= "symbol",inplace=True)
PG_df=PG_df.T
PG_df.head(2)

In [ ]:
T_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="T")
T_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
T_df.drop(["a"], axis=1, inplace=True)
T_df=T_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
T_df.set_index("Line Item",inplace=True)
T_df.drop(columns= "symbol",inplace=True)
T_df=T_df.T
T_df.head(2)

In [ ]:
HD_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="HD")
HD_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
HD_df.drop(["a"], axis=1, inplace=True)
HD_df=HD_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
HD_df.set_index("Line Item",inplace=True)
HD_df.drop(columns= "symbol",inplace=True)
HD_df=HD_df.T
HD_df.head(2)

In [ ]:
MA_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="MA")
MA_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
MA_df.drop(["a"], axis=1, inplace=True)
MA_df=MA_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
MA_df.set_index("Line Item",inplace=True)
MA_df.drop(columns= "symbol",inplace=True)
MA_df=MA_df.T
MA_df.head(2)

In [ ]:
INTC_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="INTC")
INTC_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
INTC_df.drop(["a"], axis=1, inplace=True)
INTC_df=INTC_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
INTC_df.set_index("Line Item",inplace=True)
INTC_df.drop(columns= "symbol",inplace=True)
INTC_df=INTC_df.T
INTC_df.head(2)

In [ ]:
DIS_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="DIS")
DIS_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
DIS_df.drop(["a"], axis=1, inplace=True)
DIS_df=DIS_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
DIS_df.set_index("Line Item",inplace=True)
DIS_df.drop(columns= "symbol",inplace=True)
DIS_df=DIS_df.T
DIS_df.head(2)

In [ ]:
VZ_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="VZ")
VZ_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
VZ_df.drop(["a"], axis=1, inplace=True)
VZ_df=VZ_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
VZ_df.set_index("Line Item",inplace=True)
VZ_df.drop(columns= "symbol",inplace=True)
VZ_df=VZ_df.T
VZ_df.head(2)

In [ ]:
UNH_df=pd.read_excel("simfin_data_bs.xlsx", sheet_name="UNH")
UNH_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
UNH_df.drop(["a"], axis=1, inplace=True)
UNH_df=UNH_df.dropna(axis='columns', how='all').dropna(axis="rows", how="any")
UNH_df.set_index("Line Item",inplace=True)
UNH_df.drop(columns= "symbol",inplace=True)
UNH_df=UNH_df.T
UNH_df.head(2)